In [168]:
import itertools
from tqdm import tqdm
from datetime import datetime, timedelta
from hs_gimme.constants.objects.req import Req
from hs_gimme.db_facade.db_facade_factory import get_mongo_client_db
from hs_fetch.utils.is_for_fetch_decider import IsForFetchDecider
from hs_recommendations_api_client import get_recommendations_api_client
from hs_gimme.logging_service.problems_only_logging_service import ProblemsOnlyLoggingService
from hs_fetch.fast_fetch.fast_fetch_employee_filtering_queries import \
    EmployeeJobTypeMatchFilter, TimeInPositionQueryFilter, \
    EmployeeJobBandsQueryFilter, \
    EmployeePerformanceRatingsQueryFilter, EmployeeIsNotRestrictedQueryFilter, EmployeeEnhancedMovesModelFeaturesFilter, \
    EmployeeIsActiveQueryFilter
from hs_fetch.fast_fetch.fast_fetch_filtering_query_builder import *
import pandas as pd

In [52]:
REQ_MAX_OPEN_DAYS = 365

In [2]:
mongo = get_mongo_client_db('production', 'jaipur')

In [10]:
audit = list(mongo.rpa_audit.find({'action_name': 'get_internal_applications_for_internal_mobility_campaign', 'context': 'RPA_V2'}))
len(audit)

52

In [26]:
talent_ids = set(itertools.chain(*[list(a.get('recommendations', {}).keys()) for a in audit]))
len(talent_ids)

259

In [33]:
apps = list(mongo.application.find({'current_talent_id': {'$in': list(talent_ids)}}, ['current_talent_id', 'samurai_json.experience']))
len(apps)

1936

In [34]:
talents_with_app = set([a['current_talent_id'] for a in apps if a['samurai_json']['experience']])
len(talents_with_app)

182

In [38]:
emps = list(mongo.employee.find({'current_talent_id': {'$in': list(talent_ids)}}))
len(emps)

261

In [40]:
emps_with_talent_card = set([e['current_talent_id'] for e in emps if e['hcm_samurai']['experience']])
len(emps_with_talent_card)

182

In [42]:
emps_with_both = emps_with_talent_card & talents_with_app
len(emps_with_both)

152

In [45]:
len(emps_with_both)/len(talent_ids), len(emps_with_talent_card)/ len(talent_ids), len(talents_with_app)/ len(talent_ids)

(0.5868725868725869, 0.7027027027027027, 0.7027027027027027)

In [80]:
req_statuses = ['Open']
minimum_req_posted_date = datetime(1971, 1, 1)

In [81]:
is_for_fetch_decider = IsForFetchDecider()

In [82]:
def _get_req_days_since_posted(req, cur_date):
    # get min of posted date internally or externally handling Nones (one of them must exist in the reqs we bring)
    date_posted = req.get(Req.DATE_POSTED)
    if req.get(Req.INTERNAL_POST_DATE):
        date_posted = min(date_posted, req.get(Req.INTERNAL_POST_DATE)) if date_posted else req.get(
            Req.INTERNAL_POST_DATE)
    return date_posted

In [306]:
countries = ['United States', 'India', 'Singapore']

In [307]:
req_max_open_days = 365

In [315]:

reqs_filter = {"$or": [{Req.DATE_POSTED: {'$exists': True}}, {Req.INTERNAL_POST_DATE: {'$exists': True}}],
               Req.REQ_STATUS: {"$in": req_statuses}}
#reqs_filter.update({Req.COUNTRY_NAME: {"$in": countries}})
reqs_filter.update({"post_status.post_internal.start_date": {"$lte": datetime.now(),
                                                             "$gte": minimum_req_posted_date}})
if req_max_open_days:
    reqs_filter.update({"open_date": {"$gte": datetime.now()-timedelta(days=req_max_open_days)}})
reqs_filter.update({"post_status.post_internal.end_date": {"$exists": False}})

cur_date = datetime.utcnow()
reqs = [req for req in mongo.req.find(reqs_filter)]
filtered_reqs = [req for req in reqs if is_for_fetch_decider.is_for_fetch(req)
                 and _get_req_days_since_posted(req, cur_date) >= minimum_req_posted_date]

# result = []
# for idx, req in enumerate(filtered_reqs):
#     if not self.should_filter_recommendation_on_applications_phases_in_req(req[Req.ID], exclude_req_phases):
#         result.append(req)

len(reqs), len(filtered_reqs)

(3029, 2886)

In [316]:
api = get_recommendations_api_client('production', logger=ProblemsOnlyLoggingService())

In [317]:
logical_filters = [
                     #   DegreeLevelQueryFilter(),
                       # CertificationsFilter(),
                       # MajorsFilter(),
                        # ProfessionsQueryFilter(),
                        # DetailedProfessionsQueryFilter(),
                        # TitlesSimilarityFeaturesFilter(filter_only_if_resume_data_exist=True,
                        #                                only_if_enough_coverage=True),
                       # MandatorySkillsQueryFilter(),
                    #   KBSkillsQueryFilter(),
                       # LanguagesQueryFilter(),
                       # VisaQueryFilter(),
                       # IndustryYearsQueryFilter(),
                       # ManagementYearsQueryFilter(),
                        # CurrentSeniorityQueryFilter(),
                        # ProfessionalSeniorityQueryFilter(),
                       # TooMuchExperienceQueryFilter(),
                       # FullTextSearchFilter(),
                       # TitlesMustIncludeQueryFilter(),
                       # IndustriesMustIncludeQueryFilter(),
                       # PastMostAdvancedPhasesFilter(),
                       # TagsQueryFilter(),
                       # CompaniesQueryFilter(),
                       # AgencyFilter(), RehireFilter(), ReferralFilter(),
                       # EmployeeJobTypeMatchFilter(TitlesSimilarityFeaturesFilter(filter_only_if_resume_data_exist=False,
                       #                                                           only_if_enough_coverage=False),
                       #                                    EmployeeEnhancedMovesModelFeaturesFilter()),
                   #     TimeInPositionQueryFilter(),
                       # EmployeeJobBandsQueryFilter(), EmployeeIsActiveQueryFilter(),
                       # EmployeeTitlesSimilarityFeaturesFilter(), EmployeeIsInPoolFilter(),
                       # EmployeeJobCategorizationNamesFilter(), EmployeeBandLevelNamesFilter(),
                       # RecentATSApplyQueryFilter(),
                       # EmployeePerformanceRatingsQueryFilter(),
                       # EmployeeIsNotRestrictedQueryFilter(),
                       # CandidateCirculationQueryFilter(),
                       # CandidateWorkTypesFilter(), CandidateRequestedSalaryFilter(), 
        # CandidateJobTypesFilter(),
       #                ExternalPoolQueryFilter()
    ]

technical_filters = [
    #            CandidateRecordQueryFilter(),
    #                      SpecificCandidateIdQueryFilter(), SpecificTalentIdsQueryFilter(),
    #                      CandidateDataSourcesFilter(), CandidateSourceSystemsFilter(),
    #                      InternalCandidatesTypeFilter(),
           #               CountryQueryFilter(), 
                          LocationQueryFilter(),
                        #  RecentApplyFromAllSourcesQueryFilter(),
    #                      CitizenshipFilter(), PopulationTypesFilter(), EmailQueryFilter(),
    #                      SearchForNameQueryFilter(), OnlySilverMedalistsFilter(), SecurityClearanceFilter()
     ]

In [318]:
filters_to_use = [f.__class__.__name__ for f in logical_filters + technical_filters] #+ ['NONE']

In [319]:
talent_to_emp = {e['current_talent_id']: e for e in emps}

In [320]:
req_ids_to_req = {r['_id']:r for r in reqs}

In [ ]:

data = []
recommendations = []
for i, f in enumerate(filters_to_use + ['ALL']):
    print(i, len(filters_to_use), f)
    additional_parameters = {"fetch_mode": {"value": "employee_fetch"},
                                 "consider_past_applied_locations_as_candidate_locations": {"value": True},
                                 "include_internals": {"value": True}, "include_externals": {"value": True},
                               # "filters_to_use": {"value": [f.__class__.__name__ for f in logical_filters]},
                             "filters_to_use": {"value": (["SpecificTalentIdsQueryFilter"] + [f]) if f != 'ALL' else []},
                                 "allow_already_applied_req": {"value": False},
                            "size": {"value": 1000},
                                 "allow_candidates_with_offer": {"value": False}}

    additional_parameters.update({'specific_talent_ids': {"value": list(talent_ids)}})



    
    for req in tqdm(filtered_reqs):
      #  if req['_id'] != 'JLLREQ280628': continue
        recommendations = api.fast_fetch('jaipur', req['_id'], **additional_parameters)['results']
        rec_talent_ids = set(r['candidate_data']['current_talent_id'] for r in recommendations)
        
        for t in talent_ids:
            data.append({
                'talent_id': t,
                'has_card': t in emps_with_talent_card,
                'has_app': t in talents_with_app,
                'req_id': req['_id'],
                'filter': f,
                'country': talent_to_emp[t]['country'],
                'is_remote': req_ids_to_req[req['_id']]['is_remote_location'],
                'recomended': t in rec_talent_ids
            })

df = pd.DataFrame(data)
len(recommendations), len(rec_talent_ids)

0 1 LocationQueryFilter


 76%|███████▋  | 2205/2886 [05:17<01:15,  9.06it/s]

In [ ]:
df.groupby(['filter', 'country', 'is_remote'])['recomended'].mean()